# SQL Generation and Analysis Workflow Test

This notebook demonstrates the complete workflow using multiple LLMs:
1. Generating SQL using DeepSeek and Mistral
2. Extracting entities from the SQL
3. Matching values in the SQL
4. Refining the generated SQL
5. Executing the refined SQL
6. Analyzing the results

Each step uses DeepSeek and Mistral for generation, with Gemini as the decision maker.

In [1]:
from llm_config.llm_call import generate_text

In [2]:
# Test queries
test_queries = [
    "Find the name and email of a learner named David Gardner or who has the email allisonthomson@example.com",
    "Find the number of students form the institution with l4g_code L4G0003",
    "Show me all learners who have completed their course in the last 6 months",
    "Analyze the distribution of students by gender, age group, and enrollment year. Skip any 'O' values in data",
    "What is the difference in count between male and female students",
    "Order the type of institution in decreasing order of gender 'M', ascending order of gender 'F' and descending order of gender 'O' students"
]

# Select which query to test
user_query = test_queries[4]

print(user_query)

What is the difference in count between male and female students


## 1. SQL Generation Test with Multiple LLMs

In [3]:
from engine.generator import SQLGenerator

def test_generator(model1="deepseek-chat", model2="mistralai/Mistral-7B-Instruct-v0.3", validator="gemini"):
    """Test SQL generation functionality with multiple LLMs"""
    generator = SQLGenerator()
    
    try:
        # Generate SQL using Model 1
        print("\nGenerating SQL with Model 1...")
        model1_results = generator.main_generator(user_query, llm_model=model1)
        print("\nGenerated SQL:", model1_results['generated_sql'])
        
        # Generate SQL using Model 2
        print("\nGenerating SQL with Model 2...")
        model2_results = generator.main_generator(user_query, llm_model=model2)
        print("\nGenerated SQL:", model2_results['generated_sql'])

        print("\nSchema Info:", model1_results['formatted_metadata'])
        
        # Use Validator to decide which SQL to use
        decision_prompt = f"""Compare these two Generated SQLs for the given query and choose the better one:
        
        Input Query: {user_query}
        
        Schema Info: {model1_results['formatted_metadata']}
        
        Query 1 (Model 1):
        Generated SQL: {model1_results['generated_sql']}
        
        Query 2 (Model 2):
        Generated SQL: {model2_results['generated_sql']}
        
        Consider the input query, schema info, and the generated SQLs when making your decision.
        Respond with only '1' or '2' to indicate which output is better."""
        
        decision = generate_text(decision_prompt, model=validator)
        
        # Use the chosen query
        if decision.strip() == '1':
            print("\nValidator chose Model 1's query")
            chosen_results = model1_results
        else:
            print("\nValidator chose Model 2's query")
            chosen_results = model2_results
        
        return chosen_results['generated_sql']
        
    except Exception as e:
        print(f"Error: {str(e)}")
        return None

generated_sql = test_generator()


Generating SQL with Model 1...

Generated SQL: SELECT 
    (SELECT COUNT(*) FROM Learner WHERE gender = 'M') - 
    (SELECT COUNT(*) FROM Learner WHERE gender = 'F') AS male_female_count_difference;

Generating SQL with Model 2...
Error: 402 - {"error":"You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits."}

Generated SQL: Error: 402 - {"error":"You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits."}

Schema Info: Country (id, name)

State (id, name, code, country_code)

District (id, name, code, state_code)

Institution (id, name, short_name, aicte_code, eamcet_code, l4g_code, l4g_group_code, type, address, website, latlong, district_code)
  - type: [Public University, Private University-State, Private University-Deemed to be, Autonomous College, Affiliated College, Unknown]

Degree (id, name, short_name)

Branch (id, name, short

## 2. Entity Extraction Test with Multiple LLMs

In [4]:
from engine.entity_extractor import EntityExtractor

def test_entity_extractor(sql_query, model1="deepseek-chat", model2="mistralai/Mistral-7B-Instruct-v0.3", validator="gemini"):
    """Test entity extraction functionality with multiple LLMs"""
    extractor = EntityExtractor()
    
    try:
        # Extract entities using Model 1
        print("\nExtracting entities with Model 1...")
        model1_results = extractor.main_entity_extractor(sql_query, llm_model=model1)
        print("\nExtracted Entities:")
        for entity in model1_results:
            print("---")
            print(f"Table: {entity['table']}")
            print(f"Column: {entity['column']}")
            print(f"Value: {entity['value']}")
        
        # Extract entities using Model 2
        print("\nExtracting entities with Model 2...")
        model2_results = extractor.main_entity_extractor(sql_query, llm_model=model2)
        print("\nExtracted Entities:")
        for entity in model2_results:
            print("---")
            print(f"Table: {entity['table']}")
            print(f"Column: {entity['column']}")
            print(f"Value: {entity['value']}")
        
        # Use Validator to decide which entities to use
        decision_prompt = f"""Compare these two sets of extracted entities for the input SQL and choose the better one:
        
        Input SQL: {sql_query}
        
        Entities 1 (Model 1):
        Extracted Entities: {model1_results}
        
        Entities 2 (Model 2):
        Extracted Entities: {model2_results}
        
        Consider the input SQL and both the extracted entities when making your decision.
        Respond with only '1' or '2' to indicate which set is better."""
        
        decision = generate_text(decision_prompt, model=validator)
        
        # Use the chosen entities
        if decision.strip() == '1':
            print("\nValidator chose Model 1's entities")
            chosen_results = model1_results
        else:
            print("\nValidator chose Model 2's entities")
            chosen_results = model2_results
        
        return chosen_results
        
    except Exception as e:
        print(f"Error: {str(e)}")
        return None

if generated_sql:
    extracted_entities = test_entity_extractor(generated_sql)
else:
    print("Skipping entity extraction as no SQL was generated")


Extracting entities with Model 1...

Extracted Entities:
---
Table: Learner
Column: gender
Value: M
---
Table: Learner
Column: gender
Value: F

Extracting entities with Model 2...
Error: 402 - {"error":"You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits."}

Extracted Entities:

Validator chose Model 1's entities


## 3. Value Matching Test with Multiple LLMs

In [5]:
from engine.value_matcher import ValueMatcher

def test_value_matcher(extracted_entities):
    """Test value matching functionality"""
    matcher = ValueMatcher()
    
    try:
        value_mappings = []
        for entity in extracted_entities:
            match = matcher.main_value_matcher(entity)
            value_mappings.extend(match)
        
        print("Value Mappings:")
        for mapping in value_mappings:
            print("---")
            print(f"Original: '{mapping['original_value']}'")
            print(f"Matched: '{mapping['matched_value']}'")
            print(f"Score: {mapping['score']}")
        
        return value_mappings
        
    except Exception as e:
        print(f"Error: {str(e)}")
        return None

if extracted_entities:
    value_mappings = test_value_matcher(extracted_entities)
else:
    print("Skipping value matching as no entities were extracted")

Value Mappings:
---
Original: 'M'
Matched: 'M'
Score: 100
---
Original: 'F'
Matched: 'F'
Score: 100


## 4. SQL Refinement Test with Multiple LLMs

In [6]:
from engine.refiner import SQLRefiner

def test_refiner(sql_query, value_mappings, model1="deepseek-chat", model2="mistralai/Mistral-7B-Instruct-v0.3", validator="gemini"):
    """Test SQL refinement functionality with multiple LLMs"""
    refiner = SQLRefiner()
    
    try:
        # Refine SQL using Model 1
        print("\nRefining SQL with Model 1...")
        model1_results = refiner.main_refiner(sql_query, value_mappings, llm_model=model1)
        print("\nRefined SQL:", model1_results['refined_sql'])
        
        # Refine SQL using Model 2
        print("\nRefining SQL with Model 2...")
        model2_results = refiner.main_refiner(sql_query, value_mappings, llm_model=model2)
        print("\nRefined SQL:", model2_results['refined_sql'])
        
        # Use Validator to decide which refined SQL to use
        decision_prompt = f"""Compare these two refined SQL queries for the given original SQL and value mappings, and choose the better one:
        
        Original SQL: {sql_query}
        Value Mappings: {value_mappings}
        
        Query 1 (Model 1):
        Refined SQL: {model1_results['refined_sql']}
        
        Query 2 (Model 2):
        Refined SQL: {model2_results['refined_sql']}
        
        Consider the original sql, value mappings and the refined SQLs when making your decision.
        Respond with only '1' or '2' to indicate which query is better."""
        
        decision = generate_text(decision_prompt, model=validator)
        
        # Use the chosen refined SQL
        if decision.strip() == '1':
            print("\nValidator chose Model 1's refined SQL")
            chosen_results = model1_results
        else:
            print("\nValidator chose Model 2's refined SQL")
            chosen_results = model2_results
        
        return chosen_results['refined_sql']
        
    except Exception as e:
        print(f"Error: {str(e)}")
        return None

if value_mappings:
    refined_sql = test_refiner(generated_sql, value_mappings)
else:
    print("Skipping refinement as no value mappings were generated")


Refining SQL with Model 1...

Refined SQL: SELECT 
    (SELECT COUNT(*) FROM Learner WHERE gender = 'M') - 
    (SELECT COUNT(*) FROM Learner WHERE gender = 'F') AS male_female_count_difference;

Refining SQL with Model 2...

Refined SQL: SELECT 
    (SELECT COUNT(*) FROM Learner WHERE gender = 'M') - 
    (SELECT COUNT(*) FROM Learner WHERE gender = 'F') AS male_female_count_difference;

Validator chose Model 1's refined SQL


## 5. SQL Execution Test

In [7]:
from engine.executor import SQLExecutor

def test_executor(sql_query):
    """Test SQLExecutor functionality"""
    executor = SQLExecutor()
    
    success, results, formatted_results, error = executor.main_executor(sql_query)
    
    if success:
        print(f"\nSuccess! Found {len(results)} rows")
        print("\nFormatted Results:")
        print(formatted_results)
        print("\nRaw Results:")
        print(results)
        return results
    else:
        print(f"\nFailed: {error}")
        return None

if refined_sql:
    execution_results = test_executor(refined_sql)
else:
    print("Skipping execution as no refined SQL was generated")


Success! Found 1 rows

Formatted Results:
male_female_count_difference
----------------------------
21                          

Raw Results:
[{'male_female_count_difference': 21}]


## 6. Results Analysis Test with Multiple LLMs

In [8]:
from engine.analyzer import SQLAnalyzer

def test_analyzer(query, results):
    """Test SQLAnalyzer functionality with DeepSeek"""
    analyzer = SQLAnalyzer()
    
    try:
        # Analyze results using DeepSeek
        print("\nAnalyzing results with DeepSeek...")
        mistral_results = analyzer.main_analyzer(query, results, llm_model="deepseek-chat")
        print("\nAnalysis:", mistral_results['analysis'])
        
    except Exception as e:
        print(f"Error: {str(e)}")

if execution_results:
    test_analyzer(user_query, execution_results)
else:
    print("Skipping analysis as no execution results were generated")


Analyzing results with DeepSeek...

Analysis: Here's a comprehensive analysis of the data for the query "What is the difference in count between male and female students":

1. Key Findings:
- There are 21 more male students than female students in the dataset
- The gender disparity is significant enough to warrant attention (21 students difference)

2. Notable Relationships:
- The single metric shows a clear gender imbalance in student population
- No other demographic breakdowns are available to analyze intersectional factors

3. Trends/Anomalies:
- The data shows a consistent pattern of male overrepresentation
- Without historical data, we cannot determine if this is an improving/worsening trend

4. Actionable Insights & Recommendations:
- Investigate root causes of gender disparity (admissions processes, institutional factors)
- Implement targeted outreach programs to attract more female applicants
- Consider gender-balanced admissions policies if appropriate for the institution
- 

## 7. LLM Model Test

In [9]:
# Define a prompt to test the conversation history
prompt = "What was our last chat about? Explain in a single sentence."
response = generate_text(prompt, model="deepseek-chat")
print(response)

Our last chat was about analyzing the difference in count between male and female students from a dataset showing 21 more males than females.
